In [4]:
from mst_algorithm import MST
from mst_graphs_local import NetworkXGraph
import networkx as nx
import random

In [5]:
size = 1000
degree = 4
max_weight = 10
max_read = 300

graph = nx.generators.random_regular_graph(degree, size)

for (u, v) in graph.edges():
    graph.edges[u, v]['weight'] = random.randint(0, max_weight)

approx = MST(
    NetworkXGraph(graph, size, max_weight, max_read)
).approx_weight()

print("Our MST weight approx:", approx)

TypeError: '<=' not supported between instances of 'dict' and 'int'